In [1]:
import numpy as np
import pandas as pd
from usz_neuro_conversion.common import read_nix, NixContext

Read the data we already have from existing nix files. The mapping from verbal to visual participant comes from "../in/metadata/participant_mapping.csv"

In [11]:
verbal_to_visual = {
    6: 1,
    8: 3,
    4: 4,
    3: 9,
    7: 10,
    1: 12,
    2: 13,
}
visual_locations = pd.DataFrame(columns=["participant", "electrode", "X", "Y", "Z", "location"])
for verbal_i, visual_i in verbal_to_visual.items():
    nix_context = NixContext(
        session=1, subject=verbal_i, project="Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM"
    )
    nixfile = read_nix(nix_context)
    _general = nixfile.sections["General"]

    session = nixfile.blocks[f"Data_Subject_0{verbal_i}_Session_01"]
    _electrode_map_ieeg = (
        session
        .groups["iEEG electrode information"]
        .data_arrays["iEEG_Electrode_MNI_Coordinates"]
    )

    ieeg_electrode_locations = np.ndarray(_electrode_map_ieeg.shape)
    _electrode_map_ieeg.read_direct(ieeg_electrode_locations)

    ieeg_electrode_locations_df = pd.DataFrame(
        ieeg_electrode_locations, columns=["X", "Y", "Z"]
    )
    _labels = (
        session
        .groups["iEEG data"]
        .data_arrays["iEEG_Data_Trial_01"]
        .dimensions[0]
        .labels
    )
    ieeg_electrode_locations_df.insert(3, "electrode", _labels)
    ieeg_electrode_locations_df.insert(0, "participant", visual_i)

    ieeg_electrode_information = (
        session
        .groups["iEEG electrode information"]
        .data_arrays
    )

    names_and_locations = []
    for source in ieeg_electrode_information["iEEG_Electrode_Map"].sources:
        electrode_name = source.sources[0].name
        location = source.sources[1].name
        names_and_locations.append([electrode_name, location])
    names_and_locations = pd.DataFrame(names_and_locations, columns=["electrode", "location"])
    ieeg_electrode_locations_df = ieeg_electrode_locations_df.merge(names_and_locations, on="electrode")

    visual_locations = pd.concat([visual_locations, ieeg_electrode_locations_df])
print(visual_locations.to_string())

   participant electrode          X          Y          Z                                                                         location
0            1     mAHL1 -24.985748 -11.907035 -17.444419                                Hipp, Left Hippocampus rHipp, rostral hippocampus
1            1     mAHL2 -28.303377 -14.991270 -15.678862                                Hipp, Left Hippocampus rHipp, rostral hippocampus
2            1     mAHL3 -31.949166 -17.267229 -13.824129                                 Hipp, Left Hippocampus cHipp, caudal hippocampus
3            1     mAHL4 -36.525137 -20.475806 -11.981470                                 Hipp, Left Hippocampus cHipp, caudal hippocampus
4            1     mAHL5 -40.884593 -23.053460 -10.330847                                                                   no_label_found
5            1     mAHL6 -45.876287 -25.329025  -8.826048          MTG, Left Middle Temporal Gyrus aSTS, anterior superior temporal sulcus
6            1     mAHL7 -4

Read the data we don't already have from the excel file

In [25]:
from dataclasses import dataclass


@dataclass(frozen=True)
class ExcelMetadata:
    visual_participant: int
    initials: str
    cols: str
    rows: str


excel_metadata = [
    ExcelMetadata(
        visual_participant=2,
        initials="SP",
        cols="E:G,J:K",
        rows="2:33"
    ),
    ExcelMetadata(
        visual_participant=5,
        initials="PN",
        cols="A:E",
        rows="2:33"
    ),
    ExcelMetadata(
        visual_participant=6,
        initials="SA",
        cols="B:D,J:K",
        rows="2:65"
    ),
    ExcelMetadata(
        visual_participant=7,
        initials="NP_2018",
        cols="B:D,A,F",
        rows="2:25"
    ),
    ExcelMetadata(
        visual_participant=8,
        initials="DG",
        cols="A:C,M,O",
        rows="2:65"
    ),
    ExcelMetadata(
        visual_participant=11,
        initials="WT",
        cols="A:E",
        rows="2:41"
    ),
]

In [29]:
path = "../in/metadata/electrode_locations_mr_ct.xlsx"
excel = pd.DataFrame()
for metadata in excel_metadata:
    skiprows = int(metadata.rows.split(":")[0]) - 1
    nrows = int(metadata.rows.split(":")[1]) - skiprows
    sheet = pd.read_excel(
        io=path,
        sheet_name=metadata.initials,
        header=None,
        names=["X", "Y", "Z", "electrode", "location"],
        usecols=metadata.cols,
        skiprows=skiprows,
        nrows=nrows,

    )
    sheet.insert(0, "participant", metadata.visual_participant)
    excel = pd.concat([excel, sheet])
print(excel.to_string())

    participant          X          Y          Z  electrode                                                                                        location
0             2  29.463288 -12.128326 -16.577644    'mAHR1'                                            'Hipp, Right Hippocampus rHipp, rostral hippocampus'
1             2  32.714867 -12.743193 -15.902830    'mAHR2'                                            'Hipp, Right Hippocampus rHipp, rostral hippocampus'
2             2   37.61968 -14.488964 -15.019943    'mAHR3'                                                                                'no_label_found'
3             2  41.527917 -16.639051 -14.043265    'mAHR4'                                                                                'no_label_found'
4             2  46.905129 -18.019011 -13.058225    'mAHR5'                                                                                'no_label_found'
5             2  52.112634 -20.337926 -12.044177    'mAHR6'     

In [34]:
visual_locations = pd.concat([visual_locations, excel])
visual_locations.sort_values(by=["participant", "electrode"], inplace=True)
visual_locations.reset_index(drop=True, inplace=True)
# replace location "no_label_found" with empty string
visual_locations["location"].replace("no_label_found", "", inplace=True)
print(visual_locations.to_string())

     participant  electrode          X          Y          Z                                                                                        location
0              1      mAHL1 -24.985748 -11.907035 -17.444419                                               Hipp, Left Hippocampus rHipp, rostral hippocampus
1              1      mAHL2 -28.303377 -14.991270 -15.678862                                               Hipp, Left Hippocampus rHipp, rostral hippocampus
2              1      mAHL3 -31.949166 -17.267229 -13.824129                                                Hipp, Left Hippocampus cHipp, caudal hippocampus
3              1      mAHL4 -36.525137 -20.475806 -11.981470                                                Hipp, Left Hippocampus cHipp, caudal hippocampus
4              1      mAHL5 -40.884593 -23.053460 -10.330847                                                                                                
5              1      mAHL6 -45.876287 -25.329025  -8.8260

In [35]:
visual_locations.to_csv("../in/metadata/visual_task_electrodes.csv", index=False)